#### Raw Layer Modeling

import libraries and functions.

In [ ]:
from config import spark_session,read_csv_file
from pyspark.sql.functions import col, when, count, isnan
import matplotlib.pyplot as plt
import pandas as pd

create spark session

In [ ]:
spark = spark_session()

read raw data

In [ ]:
raw_df = read_csv_file(path="/Users/sahilnagpal/Desktop/coding/competitive-programming/Projects/WALMART_SALE_PERUSAL/input_data/WALMART_SALES_DATA.csv")

### Sanity Testing

capture null value counts

In [ ]:
raw_df\
    .select([count(when(isnan(c), c)).alias(c) for c in raw_df.columns])\
    .show()

capture duplicate value

In [ ]:
raw_df\
    .groupBy("Store","Date","Weekly_Sales","Holiday_Flag","Temperature","Fuel_Price","CPI","Unemployment")\
    .agg(count("*").alias("Count"))\
    .filter(col("Count")>1)\
    .show()

check data type mismatch

In [ ]:
def detect_type_mismatch(spark_df):
    mismatches = {}
    for column, dtype in spark_df.dtypes:
        # Determine the expected data type based on the PySpark dtype
        if 'int' in dtype:
            expected_type = int
        elif 'float' in dtype or 'double' in dtype:
            expected_type = float
        elif 'string' in dtype:
            expected_type = str
        else:
            continue  # Skipping other data types for simplicity
        
        # Check for mismatches
        mismatches[column] = spark_df.filter(~col(column).cast('string').rlike(get_regex(expected_type)))
    
    return mismatches

def get_regex(expected_type):
    if expected_type == int:
        return r'^\d+$'
    elif expected_type == float:
        return r'^\d+(\.\d+)?$'
    elif expected_type == str:
        return r'.*'  # Any string is valid
    else:
        return r'.*'  # Default regex

In [ ]:
mismatches = detect_type_mismatch(raw_df)

for column, mismatch_df in mismatches.items():
    if mismatch_df.count() > 0:
        print(f"Data type mismatches in column '{column}':")
        mismatch_df.show()
    else:
        print("Data type mismatches not found in column '{}':".format(column))

Sales Trend

In [ ]:
raw_pandas_df = raw_df.toPandas()

raw_pandas_df['Date'] = pd.to_datetime(raw_pandas_df['Date'], format='%d-%m-%Y')

# Step 5: Set the 'Date' column as the index
raw_pandas_df.set_index('Date', inplace=True)

# Step 6: Convert relevant columns to numeric types
numeric_columns = ['Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
raw_pandas_df[numeric_columns] = raw_pandas_df[numeric_columns].apply(pd.to_numeric)

# Step 7: Plot the data using Matplotlib
raw_pandas_df['Weekly_Sales'].plot(style='k.')
plt.show()


In [ ]:
raw_df\
    .groupBy(col("Holiday_Flag"))\
    .agg(count("*").alias("Count"))\
    .withColumn("Holiday_Flag", when(col("Holiday_Flag") == 1, "Holiday").otherwise("Working"))\
    .show()